<a href="https://colab.research.google.com/github/Regina-Arthur/Python-Coding-Projects/blob/main/Pytorch_tutorial/Lenet_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Importing the necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
#Data directory
data = "/content/drive/MyDrive/concrete_data"

# Transform for data
transform = transforms.Compose([
    transforms.Grayscale(1),
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# ImageFolder to load data with folder names as class names
dataset = ImageFolder(root=data, transform=transform)

# Group indices by class
class_indices = {cls: [] for cls in range(len(dataset.classes))}
for idx, (_, label) in enumerate(dataset.samples):
    class_indices[label].append(idx)

# Take the first 100 per class (sorted order)
selected_indices = []
for cls, indices in class_indices.items():
    indices.sort()  # make sure it's in consistent order
    selected_indices.extend(indices[:100])  # take first 100

# Create train subset
train = Subset(dataset, selected_indices)

# Take the 50 per class (sorted order)
selected_indices = []
for cls, indices in class_indices.items():
    indices.sort()  # make sure it's in consistent order
    selected_indices.extend(indices[500:551])

# Create test subset
val = Subset(dataset, selected_indices)

# Take the 50 per class (sorted order)
selected_indices = []
for cls, indices in class_indices.items():
    indices.sort()  # make sure it's in consistent order
    selected_indices.extend(indices[551:601])

# Create test subset
test = Subset(dataset, selected_indices)

# DataLoader
train_loader = DataLoader(train, batch_size=32, shuffle=True)
val_loader = DataLoader(val, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=True)

In [5]:
class lenet(nn.Module):
  def __init__(self, classes = 2):
    super().__init__()
    #The convolutional layers
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5)

    #The pooling layer
    self.pool = nn.AvgPool2d(kernel_size = 2, stride = 2)

    #The flattening layer
    self.flatten = nn.Flatten()

    #The fully connected layer
    self.FC1 = nn.Linear(in_features = 120, out_features = 84)
    self.FC2 = nn.Linear(in_features = 84, out_features = classes)

  #The forward pass
  def forward(self,x):
    x = self.pool(F.tanh(self.conv1(x)))
    x = self.pool(F.tanh(self.conv2(x)))
    x = self.flatten(F.tanh(self.conv3(x)))
    x = F.tanh(self.FC1(x))
    x = self.FC2(x)
    return x

In [6]:
model = lenet()
model.to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)


epochs = 10
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    train_loss = loss(outputs, labels)
    train_loss.backward()
    optimizer.step()
    running_loss += train_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  train_loss = running_loss/ len(train_loader)
  train_accuracy = 100* correct/total

  print(f"Epoch {epoch+1} / {epochs}, train loss = {train_loss}, train accuracy = {train_accuracy}")

for epoch in range(epochs):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in val_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    val_loss = loss(outputs, labels)
    running_loss += val_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  val_loss = running_loss/ len(train_loader)
  val_accuracy = 100* correct/total

  print(f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

for epoch in range(epochs):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    test_loss = loss(outputs, labels)
    running_loss += test_loss.item()
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  test_loss = running_loss/ len(train_loader)
  test_accuracy = 100* correct/total

  print(f"Epoch {epoch+1}/{epochs} | "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%"
            f"Test loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%")

Epoch 1 / 10, train loss = 0.6925430383001056, train accuracy = 50.0
Epoch 2 / 10, train loss = 0.6943778225353786, train accuracy = 50.0
Epoch 3 / 10, train loss = 0.691974231175014, train accuracy = 50.0
Epoch 4 / 10, train loss = 0.6842838185174125, train accuracy = 50.0
Epoch 5 / 10, train loss = 0.6809602720396859, train accuracy = 50.0
Epoch 6 / 10, train loss = 0.6687119432858059, train accuracy = 50.0
Epoch 7 / 10, train loss = 0.6604882904461452, train accuracy = 55.5
Epoch 8 / 10, train loss = 0.6554746287209647, train accuracy = 54.5
Epoch 9 / 10, train loss = 0.6411891409329006, train accuracy = 58.5
Epoch 10 / 10, train loss = 0.6316918134689331, train accuracy = 68.0
Epoch 1/10 | Train Loss: 0.6317, Train Acc: 68.00% | Val Loss: 0.3618, Val Acc: 56.86%
Epoch 2/10 | Train Loss: 0.6317, Train Acc: 68.00% | Val Loss: 0.3758, Val Acc: 56.86%
Epoch 3/10 | Train Loss: 0.6317, Train Acc: 68.00% | Val Loss: 0.3686, Val Acc: 56.86%
Epoch 4/10 | Train Loss: 0.6317, Train Acc: 68.00